In [6]:
from dataclasses import dataclass
from enum import StrEnum
from datetime import datetime, timedelta
import polars as pl
from collections import defaultdict, Counter
from typing import Tuple
from common.constants.column_types import (
    CPZP_SCHEMA,
    OZP_SCHEMA,
    POHLAVI_CPZP,
    TYP_UDALOSTI,
)
from common.constants.column_names import SHARED_COLUMNS, OZP_COLUMNS, CPZP_COLUMNS
import pickle
from common.constants.objects import (
    Person,
    Gender,
    AgeCohort,
    Prescription,
    PrescriptionType,
)
import matplotlib.pyplot as plt
import numpy as np
import os

pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(60)

from typing import Any
import matplotlib.dates as mdates
from common.utils import draw_chart

# CONSTANTS
POJISTOVNA = "ozp"

In [4]:
with open(f"./DATACON_data/{POJISTOVNA}_persons.pkl", "rb") as f:
    persons: list[Person] = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict

def nested_defaultdict():
    return defaultdict(lambda: defaultdict(int))

# Sample data setup
death_age_vax_dates_map = defaultdict(nested_defaultdict)
death_vax_dates_map = defaultdict(lambda: defaultdict(int))

for person in persons:
    if not person.died_at:
        continue

    for vax in person.vaccines:
        days_after_vax = (person.died_at - vax.date).days
        death_age_vax_dates_map[vax.age_cohort][vax.dose_number][days_after_vax] += 1
        death_vax_dates_map[vax.dose_number][days_after_vax] += 1

# Function to group days into weeks
def group_by_weeks(data):
    weekly_data = defaultdict(int)
    for days, count in data.items():
        week = (days // 7) + 1  # Group by 7-day intervals
        weekly_data[week] += count
    return dict(weekly_data)

# Function to draw bar chart
def draw_chart(data, x_label, y_label, dose_number):
    weeks = list(data.keys())
    death_counts = list(data.values())
    
    plt.figure(figsize=(10, 6))
    plt.bar(weeks, death_counts)
    plt.xlim(0, 100)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title( f"Number of deaths depending on weeks after vaccination for dose {dose_number}")
    plt.savefig(f"out/ozp/hve_vax_days_after_vax_{dose_number}.png")

# Plotting
for dose_number in [1, 2, 3]:
    data = death_vax_dates_map[dose_number]
    weekly_data = group_by_weeks(data)  # Group the data by weeks
    draw_chart(weekly_data, "Week after vaccination", "Number of deaths",dose_number)


TypeError: draw_chart() missing 1 required positional argument: 'dose_number'